In [1]:
import pygame
import random

FPS = 60

WINDOW_WIDTH = 400
WINDOW_HEIGHT = 420
GAME_HEIGHT = 400

PADDLE_WIDTH = 15
PADDLE_HEIGHT = 60
PADDLE_BUFFER = 15

BALL_WIDTH = 20
BALL_HEIGHT = 20

PADDLE_SPEED = 3
BALL_X_SPEED = 2
BALL_Y_SPEED = 2

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))

def drawPaddle(switch, paddleYPos):
    if switch == "left":
        paddle = pygame.Rect(PADDLE_BUFFER, paddleYPos, PADDLE_WIDTH, PADDLE_HEIGHT) 
    elif switch == "right":
        paddle = pygame.Rect(WINDOW_WIDTH - PADDLE_BUFFER - PADDLE_WIDTH, paddleYPos, PADDLE_WIDTH, PADDLE_HEIGHT)
    
    pygame.draw.rect(screen, WHITE, paddle)

    
def drawBall(ballXPos, ballYPos):
    ball = pygame.Rect(ballXPos, ballYPos, BALL_WIDTH, BALL_HEIGHT)
    
    pygame.draw.rect(screen, WHITE, ball)
    
def updatePaddle(switch, action, paddleYPos, ballYPos):
    dft = 7.5 #delta frame time
    
    if switch == "left":
        if action == 1:
            paddleYPos = paddleYPos - PADDLE_SPEED*dft
        if action == 2:
            paddleYPos = paddleYPos + PADDLE_SPEED*dft
        
        if paddleYPos < 0:
            paddleYPos = 0
        if paddleYPos > GAME_HEIGHT - PADDLE_HEIGHT:
            paddleYPos = GAME_HEIGHT - PADDLE_HEIGHT
        
    elif switch == "right":
        if paddleYPos + PADDLE_HEIGHT/2 < ballYPos + BALL_HEIGHT/2:
            paddleYPos = paddleYPos + PADDLE_SPEED*dft
        if paddleYPos + PADDLE_HEIGHT/2 > ballYPos + BALL_HEIGHT/2:
            paddleYPos = paddleYPos - PADDLE_SPEED*dft
        if paddleYPos < 0:
            paddleYPos = 0
        if paddleYPos > GAME_HEIGHT - PADDLE_HEIGHT:
            paddleYPos = GAME_HEIGHT - PADDLE_HEIGHT
    
    return paddleYPos
  
    
def updateBall(paddle1YPos, paddle2YPos, ballXPos, ballYPos, ballXDirection, ballYDirection, DeltaFrameTime):
    dft = 7.5
    
    ballXPos = ballXPos + ballXDirection*BALL_X_SPEED*dft
    ballYPos = ballYPos + ballYDirection*BALL_Y_SPEED*dft
    
    score = -0.05
    
    if (ballXPos <= (PADDLE_BUFFER + PADDLE_WIDTH)) and ((ballYPos + BALL_HEIGHT) >= paddle1YPos) and (ballYPos <= (paddle1YPos + PADDLE_HEIGHT)) and (ballXDirection == -1):
        ballXDirection = 1
        score = 10
    elif (ballXPos <= 0):
        ballXDirection = 1
        score = -10
        return [score, ballXPos, ballYPos, ballXDirection, ballYDirection]
        
    if (ballXPos >= (WINDOW_WIDTH - PADDLE_WIDTH - PADDLE_BUFFER)) and ((ballYPos + BALL_HEIGHT) >= paddle2YPos) and (ballYPos <= (paddle2YPos + PADDLE_HEIGHT)) and (ballXDirection == 1):
        ballXDirection = -1
    elif (ballXPos >= (WINDOW_WIDTH - BALL_WIDTH)):
        ballXDirection = -1
        return [score, ballXPos, ballYPos, ballXDirection, ballYDirection]
    
    if ballYPos <= 0:
        ballYPos = 0
        ballYDirection = 1
    elif ballYPos >= GAME_HEIGHT - BALL_HEIGHT:
        ballYPos = GAME_HEIGHT - BALL_HEIGHT
        ballYDirection = -1
        
    return [score, ballXPos, ballYPos, ballXDirection, ballYDirection]
    
class PongGame:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Pong DCQL Env")
        
        self.paddle1YPos = GAME_HEIGHT/2 - PADDLE_HEIGHT/2
        self.paddle2YPos = GAME_HEIGHT/2 - PADDLE_HEIGHT/2
        
        self.ballXPos = WINDOW_WIDTH/2
        
        self.clock = pygame.time.Clock()
        
        self.GScore = 0.0
        
        self.ballXDirection = random.sample([-1, 1], 1)[0]
        self.ballYDirection = random.sample([-1, 1], 1)[0]
        
        self.ballYPos = random.randint(0,9)*(WINDOW_HEIGHT - BALL_HEIGHT)/9
        
    def initialDisplay(self):
        pygame.event.pump()
        screen.fill(BLACK)
        
        drawPaddle("left", self.paddle1YPos)
        drawPaddle("right", self.paddle2YPos)
        
        drawBall(self.ballXPos, self.ballYPos)
        
        pygame.display.flip()

    def playNextMove(self, action):
        DeltaFrameTime = self.clock.tick(FPS)
        
        pygame.event.pump()
        
        score = 0
        
        screen.fill(BLACK)
        
        self.paddle1YPos = updatePaddle("left", action, self.paddle1YPos, self.ballYPos)
        drawPaddle("left", self.paddle1YPos)
        self.paddle2YPos = updatePaddle("right", action, self.paddle2YPos, self.ballYPos)
        drawPaddle("right", self.paddle2YPos)
        
        [score, self.ballXPos, self.ballYPos, self.ballXDirection, self.ballYDirection] = updateBall(self.paddle1YPos, self.paddle2YPos, self.ballXPos, self.ballYPos, self.ballXDirection, self.ballYDirection,DeltaFrameTime)
        
        drawBall(self.ballXPos, self.ballYPos)
        
        if (score > 0.5 or score < -0.5):
            self.GScore = self.GScore*0.9 + 0.1*score
            
        ScreenImage = pygame.surfarray.array3d(pygame.display.get_surface())
        
        pygame.display.flip()
        
        return [score, ScreenImage]
        
#pg = PongGame()
#pg.initialDisplay()

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import random
import numpy as np

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Conv2D
from collections import deque

NBACTIONS = 3
IMGHEIGHT = 40
IMGWIDTH = 40
IMGHISTORY = 4

OBSERVEPERIOD = 2000
GAMMA = 0.975
BATCH_SIZE = 64

ExpReplay_CAPACITY = 2000


class Agent:
    def __init__(self):
        self.model = self.createModel()
        self.expReplay = deque()
        self.steps = 0
        self.epsion = 1.0
    
    def createModel(self):
        model = Sequential()
        model.add(Conv2D(32,kernel_size=4,strides=(2,2),input_shape=(IMGHEIGHT ,IMGWIDTH, IMGHISTORY),padding="same"))
        model.add(Activation("relu"))
        model.add(Conv2D(64,kernel_size=4,strides=(2,2),padding="same"))
        model.add(Activation("relu"))
        model.add(Conv2D(64,kernel_size=3,strides=(1,1),padding="same"))
        model.add(Activation("relu"))
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation("relu"))
        model.add(Dense(units=NBACTIONS, activation="linear"))
        model.compile(loss="mse", optimizer="adam")
        return model
        
    def findBestAct(self, s):
        if random.random() < self.epsion or self.steps < OBSERVEPERIOD:
            return random.randint(0, NBACTIONS -1)
        else:
            qvalue = self.model.predict(s)
            bestAct = np.argmax(qvalue)
            return bestAct
    
    def captureSample(self, sample):
        self.expReplay.append(sample)
        if len(self.expReplay) > ExpReplay_CAPACITY:
            self.expReplay.popleft()
        
        self.steps += 1
        self.epsion = 1.0
        if self.steps > OBSERVEPERIOD:
            self.epsion = 0.75
            if self.steps > 7000:
                self.epsion = 0.5
            if self.steps > 14000:
                self.epsion = 0.25
            if self.steps > 30000:
                self.epsion = 0.15
            if self.steps > 45000:
                self.epsion = 0.1
            if self.steps > 70000:
                self.epsion = 0.05
        
    
    def process(self):
        if self.steps > OBSERVEPERIOD:
            minibatch = random.sample(self.expReplay, BATCH_SIZE)
            batchlen = len(minibatch)
            
            inputs = np.zeros((BATCH_SIZE, IMGHEIGHT, IMGWIDTH, IMGHISTORY))
            targets = np.zeros((inputs.shape[0], NBACTIONS))
            
            Q_sa = 0
            
            for i in range(batchlen):
                state_t = minibatch[i][0]
                action_t = minibatch[i][1]
                reward_t = minibatch[i][2]
                state_t1 = minibatch[i][3]
                
                inputs[i : i +1] = state_t
                targets[i] = self.model.predict(state_t)
                Q_sa = self.model.predict(state_t1)
                
                if state_t1 is None:
                    targets[i, action_t] = reward_t
                else:
                    targets[i, action_t] = reward_t + GAMMA*np.max(Q_sa)
                    
            self.model.fit(inputs, targets, batch_size=BATCH_SIZE, epochs=1, verbose=0)

In [ ]:
import numpy as np
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.exposure import rescale_intensity
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

TOTAL_TrainTime = 100000

def processGameImage(RawImage):
    GreyImage = rgb2gray(RawImage)
    CroppedImage = GreyImage[0:400, 0:400]
    
    ReducedImage = resize(CroppedImage,(IMGHEIGHT, IMGWIDTH))
    ReducedImage = rescale_intensity(ReducedImage, out_range=(0,255))
    return ReducedImage / 128
    
def trainExperiment():
    TrainHistory = []
    TheGame = PongGame()
    TheGame.initialDisplay()
    
    TheAgent = Agent()
    BestAction = 0
    [InitialScore, InitialScreenImage] = TheGame.playNextMove(BestAction)
    InitialGameImage = processGameImage(InitialScreenImage)
   
    GameState = np.stack((InitialGameImage, InitialGameImage, InitialGameImage, InitialGameImage), axis=2)
    
    GameState = GameState.reshape(1, GameState.shape[0], GameState.shape[1], GameState.shape[2])
    
    for i in range(TOTAL_TrainTime):
        
        BestAction = TheAgent.findBestAct(GameState)
        [ReturnScore, NewScreenImage] = TheGame.playNextMove(BestAction)
        
        NewGameImage = processGameImage(NewScreenImage)
        NewGameImage = NewGameImage.reshape(1, NewGameImage.shape[0], NewGameImage.shape[1], 1)
        NextState = np.append(NewGameImage, GameState[:,:,:,:3], axis = 3)
        
        TheAgent.captureSample((GameState, BestAction, ReturnScore, NextState))
        TheAgent.process()
        
        GameState = NextState
        
        if i % 250 == 0:
            print("Train time: ",i," game score:",TheGame.GScore)
            TrainHistory.append(TheGame.GScore)

            
trainExperiment()
    



Train time:  0  game score: 0.0
Train time:  250  game score: -2.0951
Train time:  500  game score: -4.020035599
Train time:  750  game score: -1.3908908208535107
Train time:  1000  game score: -1.11640712080579
Train time:  1250  game score: -1.9841272407646109
Train time:  1500  game score: -5.2667072943990965
Train time:  1750  game score: -6.894486655855247
Train time:  2000  game score: -6.546225425415965
Train time:  2250  game score: -7.960580651453874
Train time:  2500  game score: -6.795743268876999
Train time:  2750  game score: -4.995718442839179
Train time:  3000  game score: -1.932821783312106
Train time:  3250  game score: -3.924211934827966
Train time:  3500  game score: -4.013675450440628
Train time:  3750  game score: -4.665135216730688
Train time:  4000  game score: -6.8498156941273045
Train time:  4250  game score: -6.139847669225232
Train time:  4500  game score: -2.300618650200808
Train time:  4750  game score: -0.5755923067570754
Train time:  5000  game score: -3.

Train time:  41250  game score: 9.952864293148526
Train time:  41500  game score: 9.972166836461273
Train time:  41750  game score: 9.985208315735818
Train time:  42000  game score: 9.991265658358845
Train time:  42250  game score: 9.994842458604316
Train time:  42500  game score: 9.996954523381264
Train time:  42750  game score: 9.998201676511403
Train time:  43000  game score: 8.37893810796322
Train time:  43250  game score: 7.042779163371202
Train time:  43500  game score: 4.995790668179062
Train time:  43750  game score: 7.34055798848775
Train time:  44000  game score: 5.351626086622131
Train time:  44250  game score: 7.255181687889502
Train time:  44500  game score: 3.609012234881872
Train time:  44750  game score: 6.226185634575397
Train time:  45000  game score: 5.771600355360427
Train time:  45250  game score: 7.503172293836779
Train time:  45500  game score: 6.72564820778768
Train time:  45750  game score: 8.066528010216548
Train time:  46000  game score: 8.972473712277491
Tra